In [1]:
# -*- coding: utf-8 -*-

from config import db_config
from collections import defaultdict
import pyodbc
from beem.account import Account

In [2]:
connection = pyodbc.connect(driver=db_config['driver'],
                            server=db_config['server'],
                            database=db_config['database'],
                            uid=db_config['uid'],
                            pwd=db_config['pwd'])
cursor = connection.cursor()

In [3]:
def to_sql_list(x):
    return "('" + "', '".join(x) + "')"

In [9]:
query = """\
SELECT body, COUNT(*) as cnt
FROM Comments (NOLOCK)
WHERE depth = 1 AND LEN(body) < 15 AND created BETWEEN GETUTCDATE() - 60 AND GETUTCDATE()
GROUP BY body
ORDER BY cnt DESC"""

spam_comments = [row[0] for row in cursor.execute(query)]
with open('spam_comments.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(spam_comments))

In [17]:
spam_phrases = [
    'nice', 'nice post', 'good', 'beautiful', 'good post',
    'thanks', 'upvoted', 'very nice', 'great', 'nice blog',
    'thank you', 'wow', 'amazing', 'nice one', 'awesome',
    'great post', 'lol', 'like', 'cool', 'hi',
    'nice, upvoted', 'good job', 'nice article', 'nice pic', 'nice photo',
    'welcome', 'hello', 'good article', 'nice picture', 'nice info',
    'promote me', 'fantastic', 'super', 'nice work', 'nice video',
    'good project', 'wonderful', 'nice bro', 'lovely', 'nice shot'
]

query = """\
SELECT TOP 750 author
FROM Comments (NOLOCK)
WHERE depth = 1 AND
      created BETWEEN GETUTCDATE() - 60 AND GETUTCDATE() AND
      body in """ + to_sql_list(spam_phrases) + """
GROUP BY author
ORDER BY COUNT(*) DESC"""

comment_spammers = [row[0] for row in cursor.execute(query)]
with open('comment_spammers.txt', 'w') as f:
    f.write('\n'.join(comment_spammers))

In [4]:
query1 = """\
SELECT account
FROM (
       SELECT SUBSTRING(body, CHARINDEX('@', body) + 1, CHARINDEX(' leads', body) - CHARINDEX('@', body) - 1) as account, *
       FROM Comments (NOLOCK)
       WHERE depth > 0 AND
             author = 'guard' AND CONTAINS(body, 'phishing')) C
GROUP BY account
ORDER BY COUNT(*) DESC"""

query2 = """\
 SELECT DISTINCT SUBSTRING(body, CHARINDEX('@', body) + 1, CHARINDEX(' is a', body) - CHARINDEX('@', body) - 1) as account
 FROM Comments (NOLOCK)
 WHERE depth > 0 AND
       author = 'arcange' AND 
       CONTAINS(body, 'CONFIRMED AND SCAM') AND 
       body LIKE '%The message you received from%'"""

scammers1 = [row[0] for row in cursor.execute(query1)]
scammers2 = [row[0] for row in cursor.execute(query2)]
scammers = list(set(scammers1 + scammers2))
with open('scammers.txt', 'w') as f:
    f.write('\n'.join(scammers))

In [15]:
query = """\
SELECT TOP 750 author
FROM Comments (NOLOCK)
WHERE depth = 0 AND
      category in ('utopian-io', 'dtube', 'dlive', 'steemhunt', 'polish')
ORDER BY NEWID()"""

content_creators = [row[0] for row in cursor.execute(query)]
with open('content_creators.txt', 'w') as f:
    f.write('\n'.join(content_creators))

In [18]:
bid_bots = list(map(str.strip, open('bid_bots.txt', 'r').readlines()))
full_list = content_creators + scammers + comment_spammers + bid_bots
d = defaultdict(lambda: defaultdict(int))

In [20]:
for name in full_list:
    account = Account(name)
    foll = account.get_follow_count()
    d[name]['name'] = name
    d[name]['followers'] = foll['follower_count']
    d[name]['followings'] = foll['following_count']
    d[name]['follow ratio'] = foll['following_count'] / foll['follower_count'] if foll['follower_count'] > 0 else 0
    d[name]['muters'] = len(account.get_muters())
    d[name]['reputation'] = account.get_reputation()
    d[name]['effective sp'] = account.get_steem_power()
    own_sp = account.get_steem_power(onlyOwnSP=True)
    d[name]['own sp'] = own_sp
    d[name]['sp ratio'] = account.get_steem_power() / own_sp if own_sp > 0 else 0

In [21]:
query = """\
SELECT
  name,
  curation_rewards,
  posting_rewards,
  witnesses_voted_for
FROM Accounts (NOLOCK) a
WHERE name in """ + to_sql_list(full_list)

for row in cursor.execute(query):
    name = row[0]
    curation_rewards = row[1] / 1000.0
    posting_rewards = row[2] / 1000.0
    witnesses_voted_for = row[3]   
    d[name]['curation_rewards'] = curation_rewards
    d[name]['posting_rewards'] = posting_rewards
    d[name]['witnesses_voted_for'] = witnesses_voted_for

In [22]:
query = """\
SELECT
  author,
  COUNT(*),
  AVG(LEN(body))
FROM Comments (NOLOCK) a
WHERE depth = 0 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE()
      AND author in """ + to_sql_list(full_list) + """
GROUP BY author"""

for row in cursor.execute(query):
    name = row[0]
    posts = row[1]
    average_post_len = row[2]
    d[name]['posts'] = posts
    d[name]['average_post_len'] = average_post_len

In [23]:
query="""\
SELECT
  author,
  COUNT(*),
  AVG(LEN(body)),
  CAST(SUM(CASE WHEN body LIKE '%http%' THEN 1 ELSE 0 END) as DECIMAL(10, 3)) / COUNT(*)
FROM Comments (NOLOCK) a
WHERE depth > 0 AND
      created BETWEEN GETUTCDATE() - 90 AND GETUTCDATE()
      AND author in """ + to_sql_list(full_list) + """
GROUP BY author
"""

for row in cursor.execute(query):
    name = row[0]
    comments = row[1]
    average_comment_len = row[2]
    comments_with_link_ratio = row[3]
    d[name]['comments'] = comments
    d[name]['average_comment_len'] = average_comment_len
    d[name]['comments_with_link_ratio'] = comments_with_link_ratio
    d[name]['posts_to_comments_ratio'] = d[name]['posts'] / comments if comments > 0 else 0

In [24]:
def to_class(name):
    if name in content_creators:
        return 0
    elif name in scammers:
        return 1
    elif name in comment_spammers:
        return 2
    else:
        return 3
    
for name in full_list:
    d[name]['class'] = to_class(name)

In [25]:
columns = ['name', 'followers', 'followings', 'follow ratio', 'muters',
           'reputation', 'effective sp', 'own sp', 'sp ratio', 'curation_rewards',
          'posting_rewards', 'witnesses_voted_for', 'posts', 'average_post_len', 'comments',
          'average_comment_len', 'comments_with_link_ratio', 'posts_to_comments_ratio', 'class']

with open ('data.csv' , 'w') as f:
    f.write(','.join(columns) + '\n')
    for name in full_list:
        row = [d[name][column] for column in columns]
        f.write(','.join(map(str, row)) + '\n')